## Explore the dataset

Requirements:
 - Install the `openpyxl` module to be able to read XLSX files:
    ```bash
    # Activate virtual envoironment
    conda activate dsi_participant
    # Install the module
    conda install openpyxl
    ```

### First, let's get the URLS of the dataset files
Here, we are downloading all the data available from 2014 to 2025  
It can be filtered later, after merging downloaded data

In [1]:
import json
import os
import requests

# Using https://open.toronto.ca/dataset/ttc-subway-delay-data/ as reference (For developers section)
# See https://docs.ckan.org/en/latest/api/index.html for more information

base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"
url = base_url + "/api/3/action/package_show"
params = { "id": "ttc-subway-delay-data"}
package = requests.get(url, params=params).json()
# We want the XLSX files from non-datastore_active resources for years 2022-2024
data_resource_urls = [resource["url"] for resource in package["result"]["resources"] if resource["format"] in 'XLSX' and not resource["datastore_active"] and any(year in resource["name"] for year in ["2022", "2023", "2024"])]
print('Data files URLs:\n',data_resource_urls)
ref_resource_urls = [resource["url"] for resource in package["result"]["resources"] if resource["format"] in 'XLSX' and not resource["datastore_active"] and any(refname in resource["name"] for refname in ["codes", "readme"])]
print('Reference files:\n',ref_resource_urls)

Data files URLs:
 ['https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/996cfe8d-fb35-40ce-b569-698d51fc683b/resource/441143ca-8194-44ce-a954-19f8141817c7/download/ttc-subway-delay-2022.xlsx', 'https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/996cfe8d-fb35-40ce-b569-698d51fc683b/resource/2fbec48b-33d9-4897-a572-96c9f002d66a/download/ttc-subway-delay-2023.xlsx', 'https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/996cfe8d-fb35-40ce-b569-698d51fc683b/resource/2ee1a65c-da06-4ad1-bdfb-b1a57701e46a/download/ttc-subway-delay-2024.xlsx']
Reference files:
 ['https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/996cfe8d-fb35-40ce-b569-698d51fc683b/resource/3900e649-f31e-4b79-9f20-4731bbfd94f7/download/ttc-subway-delay-codes.xlsx', 'https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/996cfe8d-fb35-40ce-b569-698d51fc683b/resource/ca43ac3d-3940-4315-889b-a9375e7b8aa4/download/ttc-subway-delay-readme.xlsx']


### Next, let's create corresponding data directories
Here, we have to have separate the reference data (e.g. delay codes and readme) from the actual data

In [2]:
# Create directories if they don't exist
# Note: path is relative to the path of the notebook
reference_dir = '../data/raw/reference-data'
delay_data_dir = '../data/raw/delay-data'
os.makedirs(reference_dir, exist_ok=True)
os.makedirs(delay_data_dir, exist_ok=True)

### Download the data
Here, we download files to the directories accordingly

In [3]:
# Download dataset files
for url in data_resource_urls + ref_resource_urls:
    filename = os.path.basename(url)
    if 'delay-codes' in filename or 'delay-readme' in filename:
        target_dir = reference_dir
    else:
        target_dir = delay_data_dir

    response = requests.get(url)
    file_path = os.path.join(target_dir, filename)

    if os.path.exists(file_path):
        print(f"File already exists: {file_path}")
    else:
        print(f"Saving file from {url} to {file_path}")
        with open(file_path, 'wb') as f:
            f.write(response.content)


File already exists: ../data/raw/delay-data/ttc-subway-delay-2022.xlsx
File already exists: ../data/raw/delay-data/ttc-subway-delay-2023.xlsx
File already exists: ../data/raw/delay-data/ttc-subway-delay-2024.xlsx
File already exists: ../data/raw/reference-data/ttc-subway-delay-codes.xlsx
File already exists: ../data/raw/reference-data/ttc-subway-delay-readme.xlsx


### Load data from files to pandas dataframe

In [4]:
import pandas as pd
import os

# Read each file from the delay-data directory
delay_data_dir = '../data/raw/delay-data'
dataframes = {}

# Iterate through each file in the directory
for filename in os.listdir(delay_data_dir):
    file_path = os.path.join(delay_data_dir, filename)
    
    # Check the file extension and read accordingly
    if filename.endswith('.xlsx'):
        df = pd.read_excel(file_path)
    elif filename.endswith('.csv'):
        df = pd.read_csv(file_path)
    else:
        continue  # Skip files that are not xlsx or csv

    # Store the dataframe in the dictionary with the filename as the key
    dataframes[filename] = df

    # Show the head of each dataframe
    print(f"{filename} head:")
    print(df.head())
    print(f"{filename} shape:", df.shape)
    print(f"{filename} info:")
    df.info()
    print(f"{filename} basic statistics:")
    df.describe()
    print(f"{filename} missing values per column:")
    print(df.isnull().sum())


ttc-subway-delay-2022.xlsx head:
        Date   Time       Day                 Station   Code  Min Delay  \
0 2022-01-01  15:59  Saturday   LAWRENCE EAST STATION   SRDP          0   
1 2022-01-01  02:23  Saturday      SPADINA BD STATION   MUIS          0   
2 2022-01-01  22:00  Saturday  KENNEDY SRT STATION TO    MRO          0   
3 2022-01-01  02:28  Saturday      VAUGHAN MC STATION   MUIS          0   
4 2022-01-01  02:34  Saturday        EGLINTON STATION  MUATC          0   

   Min Gap Bound Line  Vehicle  
0        0     N  SRT     3023  
1        0   NaN   BD        0  
2        0   NaN  SRT        0  
3        0   NaN   YU        0  
4        0     S   YU     5981  
ttc-subway-delay-2022.xlsx shape: (19895, 10)
ttc-subway-delay-2022.xlsx info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19895 entries, 0 to 19894
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       19895 non-null  date